# Day 7: Camel Cards

https://adventofcode.com/2023/day/7

In [1]:
sample_input = '''32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483'''

In [2]:
import pandas as pd

def read_input(the_input):
    lines = the_input.split('\n')
    df = pd.DataFrame(lines, columns=['string'])
    df['hand'] = df['string'].apply(lambda x: x[:5])
    df['bid'] = df['string'].apply(lambda x: int(x[6:]))
    df.drop(['string'], axis=1, inplace=True)
    return df

read_input(sample_input)
    

,hand,bid
0,32T3K,765
1,T55J5,684
2,KK677,28
3,KTJJT,220
4,QQQJA,483


In [3]:
def count_occurrences(label, hand):
    count = 0
    for i in range(len(hand)):
        if hand[i] == label:
            count += 1
    return count

count_occurrences('A', '34AA2')

2

In [43]:
def hand_type(hand):
    labels = set(hand)
    if len(labels) == 5:
        hand_type = 'High card'
        type_strength = 1
    elif len(labels) == 4:
        hand_type = 'One pair'
        type_strength = 2
    elif len(labels) == 1:
        hand_type = 'Five of a kind'
        type_strength = 7
    elif len(labels) == 2:
        max_occurrences = 1
        for label in labels:
            if count_occurrences(label, hand) == 4:
                max_occurrences = 4
        if max_occurrences == 4:
            hand_type = 'Four of a kind'
            type_strength = 6
        else:
            hand_type = 'Full House'
            type_strength = 5
    else:
        max_occurrences = 2
        for label in labels:
            if count_occurrences(label, hand) == 3:
                max_occurrences = 3
        if max_occurrences == 3:
            hand_type = 'Three of a kind'
            type_strength = 4
        else:
            hand_type = 'Two pair'
            type_strength = 3
    return hand_type, type_strength


for hand in ['32T3K', 'T55J5', 'KK677', 'KTJJT', 'QQQJA']:
    print(hand_type(hand))

('One pair', 2)
('Three of a kind', 4)
('Two pair', 3)
('Two pair', 3)
('Three of a kind', 4)


In [44]:
def card_value(card):
    value_order = '23456789TJQKA'
    return value_order.index(card)

card_value('J')

9

In [45]:
def analyze_hands(df):
    df['hand type'] = df['hand'].apply(lambda x: hand_type(x)[0])
    df['type strength'] = df['hand'].apply(lambda x: hand_type(x)[1])
    df['card1'] = df['hand'].apply(lambda x: card_value(x[0]))
    df['card2'] = df['hand'].apply(lambda x: card_value(x[1]))
    df['card3'] = df['hand'].apply(lambda x: card_value(x[2]))
    df['card4'] = df['hand'].apply(lambda x: card_value(x[3]))
    df['card5'] = df['hand'].apply(lambda x: card_value(x[4]))
    df = df.sort_values(by=['type strength', 'card1', 'card2', 'card3', 'card4', 'card5'])
    df = df.reset_index()
    df = df.drop('index', axis=1)
    df['rank'] = df.index + 1
    df['winnings'] = df['bid'] * df['rank']
    return df

In [46]:
df = read_input(sample_input)
df = analyze_hands(df)
df

,hand,bid,hand type,type strength,card1,card2,card3,card4,card5,rank,winnings
0,32T3K,765,One pair,2,1,0,8,1,11,1,765
1,KTJJT,220,Two pair,3,11,8,9,9,8,2,440
2,KK677,28,Two pair,3,11,11,4,5,5,3,84
3,T55J5,684,Three of a kind,4,8,3,3,9,3,4,2736
4,QQQJA,483,Three of a kind,4,10,10,10,9,12,5,2415


In [47]:
sum(df.winnings)

6440

In [48]:
with open('data/day07_in.txt', 'r') as f:
    puzzle_input = f.read()

In [51]:
df = read_input(puzzle_input)
df = analyze_hands(df)
df

,hand,bid,hand type,type strength,card1,card2,card3,card4,card5,rank,winnings
0,235A4,642,High card,1,0,1,3,12,2,1,642
1,23TJ9,769,High card,1,0,1,8,9,7,2,1538
2,245KT,365,High card,1,0,2,3,11,8,3,1095
3,25J79,161,High card,1,0,3,9,5,7,4,644
4,2684Q,996,High card,1,0,4,6,2,10,5,4980
...,...,...,...,...,...,...,...,...,...,...,...
995,AA2AA,709,Four of a kind,6,12,12,0,12,12,996,706164
996,AA9AA,953,Four of a kind,6,12,12,7,12,12,997,950141
997,AATAA,470,Four of a kind,6,12,12,8,12,12,998,469060
998,AAAA8,138,Four of a kind,6,12,12,12,12,6,999,137862


In [52]:
sum(df.winnings)

246409899

246409899 is correct

# Part 2

Now J is for Joker.

In [62]:
def hand_type_part2(hand):
    jokers = count_occurrences('J', hand)
    labels = set(hand)
    if len(labels) == 1:
        hand_type = 'Five of a kind'
        type_strength = 7
    elif len(labels) == 2:
        if jokers > 0:
            hand_type = 'Five of a kind'
            type_strength = 7
        else:
            max_occurrences = 1
            for label in labels:
                if count_occurrences(label, hand) == 4:
                    max_occurrences = 4
            if max_occurrences == 4:
                hand_type = 'Four of a kind'
                type_strength = 6
            else:
                hand_type = 'Full House'
                type_strength = 5
    elif len(labels) == 3:
        if jokers == 0:
            max_occurrences = 2
            for label in labels:
                if count_occurrences(label, hand) == 3:
                    max_occurrences = 3
            if max_occurrences == 3:
                hand_type = 'Three of a kind'
                type_strength = 4
            else:
                hand_type = 'Two pair'
                type_strength = 3
        else:
            max_occurrences = 1
            for label in labels:
                if count_occurrences(label, hand) > max_occurrences:
                    max_occurrences = count_occurrences(label, hand)
            if (max_occurrences == 3) or (jokers > 1):
                hand_type = 'Four of a kind'
                type_strength = 6
            else:
                hand_type = 'Full House'
                type_strength = 5
    elif len(labels) == 4:
        if jokers == 0:
            hand_type = 'One pair'
            type_strength = 2
        else:
            hand_type = 'Three of a kind'
            type_strength = 4
    else: #len(labels) == 5:
        if jokers == 0:
            hand_type = 'High card'
            type_strength = 1
        else:
            hand_type = 'One pair'
            type_strength = 2
    return hand_type, type_strength


for hand in ['32T3K', 'T55J5', 'KK677', 'KTJJT', 'QQQJA']:
    print(hand_type_part2(hand))

('One pair', 2)
('Four of a kind', 6)
('Two pair', 3)
('Four of a kind', 6)
('Four of a kind', 6)


In [63]:
def card_value_part2(card):
    value_order = 'J23456789TQKA'
    return value_order.index(card)

card_value('J')

0

In [64]:
def analyze_hands_part2(df):
    df['hand type'] = df['hand'].apply(lambda x: hand_type_part2(x)[0])
    df['type strength'] = df['hand'].apply(lambda x: hand_type_part2(x)[1])
    df['card1'] = df['hand'].apply(lambda x: card_value_part2(x[0]))
    df['card2'] = df['hand'].apply(lambda x: card_value_part2(x[1]))
    df['card3'] = df['hand'].apply(lambda x: card_value_part2(x[2]))
    df['card4'] = df['hand'].apply(lambda x: card_value_part2(x[3]))
    df['card5'] = df['hand'].apply(lambda x: card_value_part2(x[4]))
    df = df.sort_values(by=['type strength', 'card1', 'card2', 'card3', 'card4', 'card5'])
    df = df.reset_index()
    df = df.drop('index', axis=1)
    df['rank'] = df.index + 1
    df['winnings'] = df['bid'] * df['rank']
    return df

In [65]:
df = read_input(sample_input)
df = analyze_hands_part2(df)
df

,hand,bid,hand type,type strength,card1,card2,card3,card4,card5,rank,winnings
0,32T3K,765,One pair,2,2,1,9,2,11,1,765
1,KK677,28,Two pair,3,11,11,5,6,6,2,56
2,T55J5,684,Four of a kind,6,9,4,4,0,4,3,2052
3,QQQJA,483,Four of a kind,6,10,10,10,0,12,4,1932
4,KTJJT,220,Four of a kind,6,11,9,0,0,9,5,1100


In [66]:
sum(df.winnings)

5905

In [67]:
df = read_input(puzzle_input)
df = analyze_hands_part2(df)
sum(df.winnings)

244848487

244848487 is correct!